In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
import random
from IPython.display import clear_output

In [4]:
# value
keyword = "電子商務"
current_index = len(os.listdir('104data/url/')) - 1

In [5]:
# get location
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
location = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    location.append(ndf)
location = pd.concat(location, ignore_index=True)
location = location.loc[:,['city','des','no']]
location = location.sort_values('no')
location

,city,des,no
0,台北市,台北市中正區,6001001001
1,台北市,台北市大同區,6001001002
2,台北市,台北市中山區,6001001003
3,台北市,台北市松山區,6001001004
4,台北市,台北市大安區,6001001005
...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006
361,連江縣,連江縣南竿鄉,6001023001
362,連江縣,連江縣北竿鄉,6001023002
363,連江縣,連江縣莒光鄉,6001023003


In [6]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
        'Referer': 'https://www.104.com.tw/jobs/search/',
    }

In [13]:
all_jobs = []
for index, row in location.iterrows():
    print(index)
    print('start location:', row["no"])
    if index <= current_index:
        print('pass', row["no"])
        continue
    url = f'https://www.104.com.tw/jobs/search/list?'
    query = f'ro=0&kwop=7&keyword={keyword}&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area={row["no"]}&order=17&asc=0&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1'
    
    jobs = []
    page = 1
    while True:
        print('processing....', len(jobs))
        params = f'{query}&page={page}'
        r = requests.get(url, params=params, headers=headers)
        if r.status_code != requests.codes.ok:
            print('請求失敗', r.status_code)
            data = r.json()
            print(data['status'], data['statusMsg'], data['errorMsg'])
            break

        data = r.json()
        jobs.extend(job['link']['job'] for job in data['data']['list'])
        
        if (page == data['data']['totalPage']) or (data['data']['totalPage'] == 0):
            break
        
        page += 1
        time.sleep(random.uniform(3, 5))
        
    current_index = index
    
    joburl = pd.DataFrame(jobs, columns=['url'])
    joburl.to_csv(f'104data/url/{row["no"]}.csv')

0
start location: 6001001001
pass 6001001001
1
start location: 6001001002
processing.... 0
processing.... 22
processing.... 42
processing.... 62
processing.... 82
processing.... 102
processing.... 122
processing.... 142
processing.... 162
processing.... 182
processing.... 202
processing.... 222
processing.... 242
processing.... 262
processing.... 282
processing.... 302
processing.... 322
processing.... 342
processing.... 362
processing.... 382
processing.... 402
processing.... 422
processing.... 442
processing.... 462
processing.... 482
processing.... 502
processing.... 522
processing.... 542
2
start location: 6001001003
processing.... 0
processing.... 30
processing.... 50
processing.... 70
processing.... 90
processing.... 110
processing.... 130
processing.... 150
processing.... 170
processing.... 190
processing.... 210
processing.... 230
processing.... 250
processing.... 270
processing.... 290
processing.... 310
processing.... 330
processing.... 350
processing.... 370
processing.... 3

KeyboardInterrupt: 

In [ ]:
def get_job(job_id):
    """取得職缺詳細資料"""
    url = f'https://www.104.com.tw/job/ajax/content/{job_id}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
        'Referer': f'https://www.104.com.tw/job/{job_id}'
    }

    r = requests.get(url, headers=headers)
    if r.status_code != requests.codes.ok:
        print('請求失敗', r.status_code)
        return

    data = r.json()['data']
    
    x = {
        'job_name' : data['header']['jobName'],
        'cust_name' : data['header']['custName'],
        'appear_date' : data['header']['appearDate'],
        'job_detail' : data['jobDetail']['jobDescription'],
        'address_area' : data['jobDetail']['addressArea'],
        'salary_min' : data['jobDetail']['salaryMin'],
        'salary_max' : data['jobDetail']['salaryMax'],
        'work_exp' : data['condition']['workExp'],
        'major' : ' '.join(data['condition']['major']),
        'edu' : data['condition']['edu'],
        'language' : ', '.join([x['language'] + ':' + x['ability']
                            for x in data['condition']['language']]),
        'localLanguage' : ', '.join(
            [x['language'] + ':' + x['ability'] for x in data['condition']['localLanguage']]),
        'specialty' : ', '.join(x['description']
                            for x in data['condition']['specialty']),
        'skill' : ', '.join(x['description'] for x in data['condition']['skill']),
        'other' : data['condition']['other'],
    }
    res = pd.DataFrame(x, index=[0])
    return res

In [ ]:
def get_job_id(url):
    job_id = url.split('/job/')[-1]
    if '?' in job_id:
        job_id = job_id.split('?')[0]
    return job_id

dir_list = os.listdir('104data/url/')

for d in dir_list:
    df = pd.read_csv(d)
    df['id'] = df['url'].apply(get_job_id)
    
    out = pd.DataFrame(columns=['job_name','cust_name', 'appear_date','job_detail', 'address_area', 'salary_min', 'salary_max',
                       'work_exp', 'major', 'edu', 'language', 'localLanguage', 'specialty', 'skill', 'other'])
    for index, row in df.iterrows():
        res = get_job(row['id'])
        out = pd.concat([out, res])
    out.to_csv(f'104data/jobdetail/{d}')